In [0]:
from pyspark.sql import functions as F
# Definição do caminho do schema Bronze
catalogo = "medalhao"
bronze_db_name = "bronze"

def ingest_csv(nome_arquivo, nome_tabela):
   
    try:
        table_name = nome_tabela
        landing_path = f"/Volumes/medalhao/default/landing/{nome_arquivo}"

        # Leitura do arquivo CSV
        df = spark.read.csv(landing_path, header=True, inferSchema=True)

        # Validação: arquivo vazio
        if df.count() == 0:
            raise ValueError(f"O arquivo {nome_arquivo} está vazio ou não pôde ser lido.")

        # Adiciona timestamp de ingestão
        df_with_metadata = df.withColumn("ingestion_timestamp", F.current_timestamp())

        # Escrita no formato Delta
        df_with_metadata.write.format("delta").mode("overwrite").saveAsTable(f"{catalogo}.{bronze_db_name}.{table_name}")

        print(f"✅ Tabela bronze.{nome_tabela} criada com sucesso!\n")

    except Exception as e:
        print(f"Erro ao processar {nome_tabela}: {str(e)}")
#ver se dá pra passar um dicionário depois
ingest_csv("olist_customers_dataset.csv","ft_consumidores")
ingest_csv("olist_geolocation_dataset.csv","ft_geolocalizacao")
ingest_csv("olist_order_items_dataset.csv","ft_itens_pedidos")
ingest_csv("olist_order_payments_dataset.csv","ft_pagamentos_pedidos")
ingest_csv("olist_order_reviews_dataset.csv","ft_avaliacoes_pedidos")
ingest_csv("olist_orders_dataset.csv","ft_pedidos")
ingest_csv("olist_products_dataset.csv","ft_produtos")
ingest_csv("olist_sellers_dataset.csv","ft_vendedores")
ingest_csv("product_category_name_translation.csv","dm_categoria_produtos_traducao")

print("ingestão concluída")

In [0]:
import requests

data_inicio_formatada = "08-01-2024"
data_fim_formatada = "08-31-2024" 

endpoint = f"https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/CotacaoDolarPeriodo(dataInicial=@dataInicial,dataFinalCotacao=@dataFinalCotacao)?@dataInicial='{data_inicio_formatada}'&@dataFinalCotacao='{data_fim_formatada}'&$select=dataHoraCotacao,cotacaoCompra&$format=json"


dados_json = requests.get(endpoint).json()

#print(dados_json), utilizei o print pra entender como estava sendo retornado esse dicionário, então como value está retornando uma lista com os dados de cotação e data/hora de cada uma delas, utilizei como parametro na hora de criar um datraframe para organizar de maneira ordenada

df_spark = spark.createDataFrame(dados_json['value'])
#df_spark.display()

df_spark.write.format("delta").mode("overwrite").saveAsTable(f"{catalogo}.{bronze_db_name}.dm_cotacao_dolar")

